In [1]:
import numpy as np
import pandas as pd
import mlflow

In [2]:
mlflow.set_tracking_uri('http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000')
mlflow.set_experiment('Exp 5 - ML Algos with HP Tuning')

<Experiment: artifact_location='s3://neeraj-first-bucket/870835495118995563', creation_time=1736945300249, experiment_id='870835495118995563', last_update_time=1736945300249, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import ADASYN
import optuna

c:\data\projects\comment_analysis\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2)

# vectorizer
ngram_range = (1,2)
max_features = 2000
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
X_train_trf = vectorizer.fit_transform(X_train)
X_test_trf = vectorizer.fit_transform(X_test)

# oversampling
adasyn = ADASYN(random_state=42)
X_train_trf, y_train = adasyn.fit_resample(X_train_trf, y_train)


In [6]:
def log_mlflow(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_Adasyn_Tfidf_Bigram")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algorithm", model_name)

        # model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # metrics
        accuracy = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        mlflow.log_metric("accoracy", accuracy)
        for label, metrics in class_report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [7]:
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train_trf, y_train).predict(X_test_trf))

In [8]:
def optuna_run_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_svm, n_trials=30)

    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)
    
    log_mlflow("LoR", best_model, X_train_trf, y_train, X_test_trf, y_test)

In [9]:
optuna_run_experiment()

[I 2025-01-17 11:26:40,880] A new study created in memory with name: no-name-3de133a4-8725-4a99-ac9a-7314829debf7
[I 2025-01-17 11:26:41,039] Trial 0 finished with value: 0.4286103913814264 and parameters: {'C': 0.00233358755413244, 'penalty': 'l1'}. Best is trial 0 with value: 0.4286103913814264.
[I 2025-01-17 11:26:41,361] Trial 1 finished with value: 0.45083867448520387 and parameters: {'C': 0.27154666641862885, 'penalty': 'l2'}. Best is trial 1 with value: 0.45083867448520387.
[I 2025-01-17 11:26:41,457] Trial 2 finished with value: 0.32756034365198416 and parameters: {'C': 0.0013501230201704993, 'penalty': 'l2'}. Best is trial 1 with value: 0.45083867448520387.
[I 2025-01-17 11:26:41,533] Trial 3 finished with value: 0.4286103913814264 and parameters: {'C': 0.0050236412366821805, 'penalty': 'l1'}. Best is trial 1 with value: 0.45083867448520387.
[I 2025-01-17 11:26:41,604] Trial 4 finished with value: 0.4286103913814264 and parameters: {'C': 0.003862003315355495, 'penalty': 'l1'}.

🏃 View run LoR_Adasyn_Tfidf_Bigram at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/870835495118995563/runs/3ebd02dc05974cd4a4d0393cbddd8627
🧪 View experiment at: http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000/#/experiments/870835495118995563


AttributeError: 'str' object has no attribute 'items'